In [ ]:
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly as py
import pandas as pd
from chart_studio.plotly import plot, iplot

# from plotly.offline import init_notebook_mode, iplot
from tqdm import tqdm_notebook

from scvi.dataset import PowSimSynthetic, LatentLogPoissonDataset
from scvi.models import VAE, IAVAE
from scvi.inference import UnsupervisedTrainer
from scvi.utils import demultiply, make_dir_if_necessary, predict_de_genes, save_fig, load_pickle, save_pickle, has_lower_mean
from scvi_utils import estimate_de_proba, estimate_lfc_density, estimate_lfc_mean, multi_train_estimates
from R_interop import all_predictions, all_de_predictions


N_EPOCHS = 100
DELTA = 0.5
SIZES = [5, 10, 20, 30, 50, 100]
SIZE = 100
N_SIZES = len(SIZES)
DO_CLOUD = True
Q0 = 5e-2
N_TRAININGS = 5
N_PICKS = 10

np.random.seed(42)

PATH_TO_SCRIPTS = "/home/ubuntu/conquer_comparison/scripts"
DIR_PATH = 'lfc_estimates/powsimr2'
make_dir_if_necessary(DIR_PATH)

# Generate Dataset

In [ ]:
import chart_studio.plotly as py
py.sign_in("pierreboyeau", "2wvdnWZ2Qut1zD07ADVy")

In [ ]:
dataset_path = os.path.join(DIR_PATH, "dataset.pickle")
if not os.path.exists(dataset_path):
    dataset = PowSimSynthetic(
        cluster_to_samples=[7500, 7500],
        de_p=0.5,
        n_genes=1500,
        mode="NB"
    )
    save_pickle(dataset, filename=dataset_path)
else:
    dataset = load_pickle(filename=dataset_path)

is_significant_de = np.abs(dataset.lfc[:, 1] - dataset.lfc[:, 0]) >= DELTA
n_genes = dataset.nb_genes
trace1 = go.Histogram(x=dataset.lfc[:, 1] - dataset.lfc[:, 0])
fig = go.Figure(data=[trace1])
# save_fig(fig, filename="powsimR_properties", do_cloud=DO_CLOUD)
# fig.show()
iplot(fig, filename="powsimR_properties")

In [ ]:
n_examples = len(dataset)
TEST_INDICES = np.random.permutation(n_examples)[:2000]

x_test, y_test = dataset.X[TEST_INDICES, :], dataset.labels[TEST_INDICES, :].squeeze()
data_path = os.path.join(DIR_PATH, 'data.npy')
labels_path = os.path.join(DIR_PATH, 'labels.npy')

np.save(
    data_path,
    x_test.squeeze().astype(int)
)
np.savetxt(
    labels_path,
    y_test.squeeze()
)

In [ ]:
np.where(y_test == 1)[0][:100]

## Train parameters

In [ ]:
mdl_params = dict(
    iaf=dict(n_hidden=128, n_layers=1, do_h=True, n_latent=10, t=4, dropout_rate=0.3),
    iaf_b=dict(n_hidden=128, n_layers=2, do_h=False, n_latent=10, t=3, dropout_rate=0.3),
    mf=dict(n_hidden=128, n_layers=1, n_latent=10, dropout_rate=0.3),
    iaf_k5=dict(n_hidden=128, n_layers=1, do_h=True, n_latent=10, t=4),
    mf_k5=dict(n_hidden=128, n_layers=1, n_latent=10),
)
train_params = dict(
    iaf=dict(ratio_loss=True, test_indices=TEST_INDICES),
    iaf_b=dict(ratio_loss=True, test_indices=TEST_INDICES),
    mf=dict(ratio_loss=True, test_indices=TEST_INDICES),
    iaf_k5=dict(ratio_loss=True, test_indices=TEST_INDICES, k_importance_weighted=5, single_backward=True),
    mf_k5=dict(ratio_loss=True, test_indices=TEST_INDICES, k_importance_weighted=5, single_backward=True)
)
train_fn_params = dict(
    iaf=dict(n_epochs=N_EPOCHS, lr=1e-3),
    iaf_b=dict(n_epochs=N_EPOCHS, lr=1e-3),
    mf=dict(n_epochs=N_EPOCHS, lr=1e-3),
    iaf_k5=dict(n_epochs=N_EPOCHS, lr=1e-3),
    mf_k5=dict(n_epochs=N_EPOCHS, lr=1e-3),
)

In [ ]:
data_path

# Compute competitors scores

In [ ]:
other_predictions = all_predictions(
    filename=os.path.join(DIR_PATH, "other_predictions_final.pickle"),
    n_genes=n_genes, 
    n_picks=N_PICKS, 
    sizes=SIZES, 
    data_path=data_path, 
    labels_path=labels_path,
    path_to_scripts=PATH_TO_SCRIPTS
)

other_predictions = all_de_predictions(
    other_predictions, significance_level=Q0, delta=DELTA
)

Check sign of LFC 

In [ ]:
other_predictions["edger"]["lfc"].shape

In [ ]:
from scvi.utils import plot_identity

lfc_gt = -(dataset.lfc[:, 1] - dataset.lfc[:, 0])
plt.scatter(lfc_gt, -other_predictions["edger"]["lfc"][-1, -1, :])
plot_identity()
plt.show()

plt.scatter(lfc_gt, other_predictions["deseq2"]["lfc"][-1, -1, :])
plot_identity()
plt.show()


plt.scatter(lfc_gt, -other_predictions["mast"]["lfc"][-1, -1, :])
plot_identity()
plt.show()


In [ ]:
other_predictions["edger"]["lfc"] = -other_predictions["edger"]["lfc"]
other_predictions["mast"]["lfc"] = -other_predictions["mast"]["lfc"]

# Experiments

In [ ]:
os.listdir(DIR_PATH)

In [ ]:
# res_mf = multi_train_estimates(
#     filename=os.path.join(DIR_PATH, "res_mf1.pickle"),
#     mdl_class=VAE,
#     dataset=dataset,
#     mdl_params=mdl_params["mf"],
#     train_params=train_params["mf"],
#     train_fn_params=train_fn_params["mf"],
#     sizes=SIZES,
#     n_trainings=N_TRAININGS,
#     n_picks=N_PICKS,
#     n_samples=500,
#     label_a=0,
#     label_b=1
# ).assign(algorithm="MF")

# res_iaf = multi_train_estimates(
#     filename=os.path.join(DIR_PATH, "res_iaf1.pickle"),
#     mdl_class=IAVAE,
#     dataset=dataset,
#     mdl_params=mdl_params["iaf"],
#     train_params=train_params["iaf"],
#     train_fn_params=train_fn_params["iaf"],
#     sizes=SIZES,
#     n_trainings=N_TRAININGS,
#     n_picks=N_PICKS,
#     n_samples=500,
#     label_a=0,
#     label_b=1
# ).assign(algorithm="IAF")

res_mf = multi_train_estimates(
    filename=os.path.join(DIR_PATH, "res_mf.pickle"),
    mdl_class=VAE,
    dataset=dataset,
    mdl_params=mdl_params["mf"],
    train_params=train_params["mf"],
    train_fn_params=train_fn_params["mf"],
    sizes=SIZES,
    n_trainings=N_TRAININGS,
    n_picks=N_PICKS,
    n_samples=500,
    label_a=0,
    label_b=1
).assign(algorithm="MF")

res_iaf = multi_train_estimates(
    filename=os.path.join(DIR_PATH, "res_iaf.pickle"),
    mdl_class=IAVAE,
    dataset=dataset,
    mdl_params=mdl_params["iaf"],
    train_params=train_params["iaf"],
    train_fn_params=train_fn_params["iaf"],
    sizes=SIZES,
    n_trainings=N_TRAININGS,
    n_picks=N_PICKS,
    n_samples=500,
    label_a=0,
    label_b=1
).assign(algorithm="IAF")

# res_mfk5 = multi_train_estimates(
#     filename=os.path.join(DIR_PATH, "res_mf_k5_final1.pickle"),
#     mdl_class=VAE,
#     dataset=dataset,
#     mdl_params=mdl_params["mf_k5"],
#     train_params=train_params["mf_k5"],
#     train_fn_params=train_fn_params["mf_k5"],
#     sizes=[SIZE],
#     n_picks=N_PICKS,
#     n_trainings=N_TRAININGS
# ).assign(algorithm="MF K5")

# res_iak5 = multi_train_estimates(
#     filename=os.path.join(DIR_PATH, "res_ia_k5_final1.pickle"),
#     mdl_class=IAVAE,
#     dataset=dataset,
#     mdl_params=mdl_params["iaf_k5"],
#     train_params=train_params["iaf_k5"],
#     train_fn_params=train_fn_params["iaf_k5"],
#     sizes=[SIZE],
#     n_picks=N_PICKS,
#     n_trainings=1
# ).assign(algorithm="IAF K5")

## FDR / Power Control and PR Curves

In [ ]:
def train_model(
    mdl_class, dataset, mdl_params: dict, train_params: dict, train_fn_params: dict
):
    """

    :param mdl_class: Class of algorithm
    :param dataset: Dataset
    :param mdl_params:
    :param train_params:
    :param train_fn_params:
    :return:
    """
    my_vae = mdl_class(dataset.nb_genes, n_batch=dataset.n_batches, **mdl_params)
    my_trainer = UnsupervisedTrainer(my_vae, dataset, **train_params)
    print(my_trainer.test_set.data_loader.sampler.indices)
    my_trainer.train(**train_fn_params)
    print(my_trainer.train_losses)
    return my_vae, my_trainer

### FDR and TPR Control

In [ ]:
def fdr_fnr(my_df):
    my_df = my_df.sort_values("gene")
    assert len(my_df) == n_genes
    is_pred_de = predict_de_genes(my_df.de_proba.values, desired_fdr=Q0)
    
    alpha = my_df.de_proba.values[is_pred_de].min()
#     alpha = 0.8
#     is_pred_de = my_df.de_proba.values >= 0.4
    true_fdr = ((1.0 - is_significant_de) * is_pred_de).sum() / is_pred_de.sum()
    n_positives = is_significant_de.sum()
    true_fnr = (is_significant_de * (1.0 - is_pred_de)).sum() / n_positives
    return pd.Series(dict(fdr=true_fdr, fnr=true_fnr, alpha=alpha))


fdr_fnr_mf = (
    res_mf.groupby(["experiment", "training", "sample_size"])
    .apply(fdr_fnr)
    .reset_index()
    .assign(algorithm="MF")
)
fdr_fnr_iaf = (
    res_iaf.groupby(["experiment", "training", "sample_size"])
    .apply(fdr_fnr)
    .reset_index()
    .assign(algorithm="IAF")
)

df = pd.concat([fdr_fnr_mf, fdr_fnr_iaf], ignore_index=True)


fig = px.box(
    df,
    x="sample_size",
    y="fdr",
    color="algorithm",
    title="Control on False Discovery Rate",
)
fig.show()
# iplot(fig, filename="powsimr_fdr_control")

fig = px.box(
    df,
    x="sample_size",
    y="fnr",
    color="algorithm",
    title="Control on False Negative Rate",
)
fig.show()
# iplot(fig, filename="powsimr_power_control")

Alpha comparisons

In [ ]:
fdr_fnr_iaf.groupby("sample_size").alpha.mean()

In [ ]:
fdr_fnr_mf.groupby("sample_size").alpha.mean()

Other algorithms

In [ ]:
##

In [ ]:
# ['deseq2', 'edger', 'mast']

def get_fdr_fnr(y_pred, y_true):
    """
        y_pred: (n_sz, n_picks, n_genes) bool predictions
        y_true: (n_genes) gt vals
    """
    n_sz, n_picks, _ = y_pred.shape
    fnrs = np.zeros((n_sz, n_picks))
    fdrs = np.zeros((n_sz, n_picks))
    for sz in range(n_sz):
        for pick in range(n_picks):
            y_pred_it = y_pred[sz, pick, :]
            fnr = ((~y_true) * y_pred_it).sum() / y_pred_it.sum()
            fdr = (y_true * (~y_pred_it)).sum() / y_true.sum()
            fnrs[sz, pick] = fnr
            fdrs[sz, pick] = fdr
    fnrs[np.isnan(fnrs)] = 0.0
    return dict(fnr=fnrs, fdr=fdrs)

print(other_predictions["mast"]['pval'].shape)
print(other_predictions["deseq2"]['pval'].shape)
print(other_predictions["edger"]['pval'].shape)

is_de_mast = other_predictions["mast"]["is_de"]
is_de_deseq2 = other_predictions["deseq2"]["is_de"]
is_de_edger = other_predictions["edger"]["is_de"]
# is_de_edgerr = other_predictions["edger_robust"]["is_de"]


res_mast = get_fdr_fnr(is_de_mast, y_true=is_significant_de)
res_deseq2 = get_fdr_fnr(is_de_deseq2, y_true=is_significant_de)
res_edger = get_fdr_fnr(is_de_edger, y_true=is_significant_de)
# res_edgerr = get_fdr_fnr(is_de_edgerr, y_true=is_significant_de)

In [ ]:
preds_mf = res_mf[(res_mf.experiment == 0) & (res_mf.training == 0) & (res_mf.sample_size == 100)]
preds_iaf = res_iaf[(res_iaf.experiment == 0) & (res_iaf.training == 0) & (res_iaf.sample_size == 100)]

# preds_mf = preds_mf.sort_values("de_proba").set_index("gene")
# preds_iaf = preds_iaf.set_index("gene").reindex(index=preds_mf.index)
# preds_iaf[]

preds = pd.concat([preds_mf, preds_iaf], ignore_index=True)
preds.head()

In [ ]:
preds_mf = preds_mf.assign(
    de_proba_iaf=preds_iaf.de_proba,
    gene_mean=dataset.X.mean(0),
    is_de=is_significant_de.astype(float),
)

import plotly.figure_factory as ff

fig = ff.create_distplot(
    [preds_mf["de_proba"], preds_mf["de_proba_iaf"]],
    ["de_proba", "de_proba_iaf"],
    bin_size=5e-2,
)
fig.show()

In [ ]:
import plotly.figure_factory as ff

fig = ff.create_distplot(
    [preds_mf["de_proba"], preds_mf["de_proba_iaf"]],
    ["de_proba", "de_proba_iaf"],
    bin_size=5e-2,
)
fig.show()

#### Tables

In [ ]:
def algos_comparison(my_df, key1, other_keys, key_values="error"):
    vals_key1 = my_df.loc[my_df["algorithm"]==key1, key_values].values
    algo1_is_better = True
    for key2 in other_keys:
        vals_other = my_df.loc[my_df["algorithm"] == key2, key_values].values
        try:
            key1_better = has_lower_mean(vals_key1, vals_other)
        except ValueError:
            key1_better = False
            break
        if not key1_better:
            algo1_is_better = False
            break
    return key1_better


gped = df.groupby("sample_size")
fdr_mf_better = gped.apply(algos_comparison, key1="MF", other_keys=["IAF"], key_values="fdr")
fdr_iaf_better = gped.apply(algos_comparison, key1="IAF", other_keys=["MF"], key_values="fdr")

fnr_mf_better = gped.apply(algos_comparison, key1="MF", other_keys=["IAF"], key_values="fnr")
fnr_iaf_better = gped.apply(algos_comparison, key1="IAF", other_keys=["MF"], key_values="fnr")

In [ ]:
res_table = df.groupby(["sample_size", "algorithm"])["fdr", "fnr"].mean().round(3).reset_index()

res_table.loc[res_table["algorithm"] == "MF", "fdr_better"] = fdr_mf_better.values
res_table.loc[res_table["algorithm"] == "IAF", "fdr_better"] = fdr_iaf_better.values
res_table.loc[res_table["algorithm"] == "MF", "fnr_better"] = fnr_mf_better.values
res_table.loc[res_table["algorithm"] == "IAF", "fnr_better"] = fnr_iaf_better.values

res_table.loc[res_table["fdr_better"], "fdr"] = res_table.loc[
    res_table["fdr_better"], "fdr"
].apply(lambda x: "\mathbf{{ {} }}".format(x))

res_table.loc[res_table["fnr_better"], "fnr"] = res_table.loc[
    res_table["fnr_better"], "fnr"
].apply(lambda x: "\mathbf{{ {} }}".format(x))

res_table.loc[:, "fdr"] = res_table.loc[:, "fdr"].apply(lambda x: "$ {} $".format(x))
res_table.loc[:, "fnr"] = res_table.loc[:, "fnr"].apply(lambda x: "$ {} $".format(x))

In [ ]:
res_table.pivot(
    index="algorithm", columns="sample_size", values=["fdr", "fnr"]
).T

In [ ]:
res_table.loc[lambda x: x["sample_size"].isin([5, 20, 100])].pivot(
    index="algorithm", columns="sample_size", values=["fdr", "fnr"]
).T

In [ ]:
print(
    res_table.loc[lambda x: x["sample_size"].isin([5, 20, 100])]
    .pivot(index="algorithm", columns="sample_size", values=["fdr", "fnr"])
    .T
    .to_latex(escape=False)
)

In [ ]:
print(res_table.pivot(index="algorithm", columns="sample_size", values="fdr").loc[
    :, [5, 20, 100]
].to_latex(escape=True))

In [ ]:
res_table.pivot(index="algorithm", columns="sample_size", values="fnr").loc[:, [5, 20, 100]]

### PR Curves

#### PR Curve

In [ ]:
selected_training = 0

preds_md = res_mf.loc[
    lambda x: (x.experiment == 0) & (x.training == selected_training) & (x.sample_size == 100)
].sort_values("gene")["de_proba"]

preds_iaf = res_iaf.loc[
    lambda x: (x.experiment == 0) & (x.training == selected_training) & (x.sample_size == 100)
].sort_values("gene")["de_proba"]

In [ ]:
from sklearn.metrics import precision_recall_curve

preds_deseq2 = 1.0 - other_predictions['deseq2']['pval'][-1, 0, :]
preds_edger = 1.0 - other_predictions['edger']['pval'][-1, 0, :]
# preds_edgerr = 1.0 - other_predictions['edger_robust']['pval'][-1, 0, :]
preds_mast = 1.0 - other_predictions['mast']['pval'][-1, 0, :]

In [ ]:
print(np.isnan(preds_md).mean())
print(np.isnan(preds_iaf).mean())
print(np.isnan(preds_deseq2).mean())
print(np.isnan(preds_deseq2).mean())
print(np.isnan(preds_edger).mean())
print(np.isnan(preds_mast).mean())

In [ ]:
preds_deseq2[np.isnan(preds_deseq2)] = 0.0

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

def plot_pr(fig, preds, y_true, name):
    average_precision = average_precision_score(y_true, preds)
    preds[np.isnan(preds)] = np.min(preds[~np.isnan(preds)])
    precs, recs, _ = precision_recall_curve(y_true=y_true, probas_pred=preds)
    fig.add_trace(
        go.Scatter(
            x=recs,
            y=precs,
            name=name+'@AP: {0:0.2f}'.format(average_precision)
        )
    )
    return
layout = go.Layout(
    title='Precision Recall Curves',
    xaxis=dict(title='Recall'),
    yaxis=dict(title='Precision'),
    width=800,
    height=600,
)
fig = go.Figure(layout=layout)
plot_pr(fig=fig, preds=preds_md, y_true=is_significant_de, name='MF')
plot_pr(fig=fig, preds=preds_iaf, y_true=is_significant_de, name='IAF')
plot_pr(fig=fig, preds=preds_deseq2, y_true=is_significant_de, name='DESeq2')
plot_pr(fig=fig, preds=preds_edger, y_true=is_significant_de, name='EdgeR')
# plot_pr(fig=fig, preds=preds_edgerr, y_true=is_significant_de, name='EdgeR Robust')
plot_pr(fig=fig, preds=preds_mast, y_true=is_significant_de, name='MAST')

fig.show()
iplot(fig, filename="powsimr_pr_curves2", sharing="private")

#### MAP

In [ ]:
def do_ap(my_df):
    my_df = my_df.sort_values("gene")
    average_precision = average_precision_score(is_significant_de, my_df.de_proba)
    return pd.Series(dict(AP=average_precision))


ap_mf = (
    res_mf.groupby(["experiment", "training", "sample_size"])
    .apply(do_ap)
    .reset_index()
    .assign(algorithm="MF")
)
ap_iaf = (
    res_iaf.groupby(["experiment", "training", "sample_size"])
    .apply(do_ap)
    .reset_index()
    .assign(algorithm="IAF")
)

all_ap = pd.concat([ap_mf, ap_iaf], ignore_index=True)

In [ ]:
px.box(all_ap, x="sample_size", y="AP", color="algorithm")

In [ ]:
# all_ap.groupby(["algorithm", "sample_size"]).agg(dict(AP=["mean", "std"]))


## Diagonal Curve

In [ ]:
lfc_gt = -(dataset.lfc[:, 1] - dataset.lfc[:, 0])

In [ ]:
res_mf.head()

In [ ]:
subsample_genes = np.sort(np.random.permutation(n_genes)[:150])

lfcs_mf = (
    res_mf
    .loc[
        lambda x: (x.experiment == 0)
        & (x.training == selected_training)
        & (x.sample_size == 100)
        & (x.gene.isin(subsample_genes))
    ]
    .sort_values("gene")
    [["lfc_mean", "hdi99_low", "hdi99_high", "algorithm"]]
    .assign(
        err_minus=lambda x: x.lfc_mean - x.hdi99_low,
        err_pos=lambda x: x.hdi99_high - x.lfc_mean,
        lfc_gt=lfc_gt[subsample_genes]
    )
)

lfcs_ia = (
    res_iaf
    .loc[
        lambda x: (x.experiment == 0)
        & (x.training == selected_training)
        & (x.sample_size == 100)
        & (x.gene.isin(subsample_genes))
    ]
    .sort_values("gene")
    [["lfc_mean", "hdi99_low", "hdi99_high", "algorithm"]]
    .assign(
        err_minus=lambda x: x.lfc_mean - x.hdi99_low,
        err_pos=lambda x: x.hdi99_high - x.lfc_mean,
        lfc_gt=lfc_gt[subsample_genes]
    )
)


all_lfcs = pd.concat([lfcs_mf, lfcs_ia], ignore_index=True)

In [ ]:
fig = px.scatter(
    all_lfcs,
    x="lfc_gt",
    y="lfc_mean",
    color="algorithm",
    error_y="err_pos",
    error_y_minus="err_minus",
)

fig.add_trace(
    go.Scatter(
        x=[-3, 3],
        y=[-3, 3],
        mode="lines",
        line=dict(color="black", width=4, dash="dash"),
    )
)

fig.show()

### Counts

In [ ]:
def frac_inside(my_df):
    confidence = 64
    is_in_hdi64 = (lfc_gt <= my_df["hdi{}_high".format(confidence)]) & (
        lfc_gt >= my_df["hdi{}_low".format(confidence)]
    )
    confidence = 99
    is_in_hdi99 = (lfc_gt <= my_df["hdi{}_high".format(confidence)]) & (
        lfc_gt >= my_df["hdi{}_low".format(confidence)]
    )
    return pd.Series([
#         (0.64 - is_in_hdi64.mean())**2, 
#         (0.99 - is_in_hdi99.mean())**2
        is_in_hdi64.mean(), 
        is_in_hdi99.mean()
    ], index=["error64", "error99"])


errs_mf = (
    res_mf.groupby(by=["experiment", "sample_size", "training"])
    .apply(frac_inside)
    .reset_index()
).assign(algorithm="Mean Field")

errs_iaf = (
    res_iaf.groupby(by=["experiment", "sample_size", "training"])
    .apply(frac_inside)
    .reset_index()
).assign(algorithm="IAF")

In [ ]:
display(errs_mf)
display(errs_iaf)

In [ ]:
def scoring(my_df):
    err_scores = my_df.loc[:, "error64"] + my_df.loc[:, "error99"]
    err_scores_mf = err_scores[my_df["algorithm"] == "Mean Field"].values
    err_scores_iaf = err_scores[my_df["algorithm"] == "IAF"].values
    
    disp_mf = str(err_scores_mf.mean().round(3))
    disp_iaf = str(err_scores_iaf.mean().round(3))
#     print(err_scores_mf)
    if has_lower_mean(err_scores_mf, err_scores_iaf):
        disp_mf = "\mathbf{{ {} }}".format(disp_mf)
    if has_lower_mean(err_scores_iaf, err_scores_mf):
        disp_iaf = "\mathbf{{ {} }}".format(disp_iaf)
    
    return pd.Series(
        dict(
            IAF=disp_mf, 
            MF=disp_iaf
        )
    )
    
    
errs = pd.concat([errs_mf, errs_iaf], ignore_index=True)
errs.groupby("sample_size").apply(scoring)


## Study of LFC errors

In [ ]:
def compute_l2_err(diff):
    res = 0.5 * (diff ** 2) ** (0.5)
    res = np.nanmean(res, axis=-1)
    return res

def l2_err_competitor(vals: np.ndarray, other: np.ndarray = None):
    vals[np.isnan(vals)] = 0.0
    if other is None:
        diff = vals
    else:
        diff = vals - other
    res = compute_l2_err(diff)
    assert res.shape == (N_SIZES, N_PICKS)
    data = []
    for (size_ix, size) in enumerate(SIZES):
        for pick in range(N_PICKS):
            data.append(dict(experiment=pick, training=0, sample_size=size, error=res[size_ix, pick]))
    return pd.DataFrame(data)

lfcs_errs_deseq2 = l2_err_competitor(other_predictions["deseq2"]["lfc"], other=lfc_gt).assign(algorithm="DESeq2")
lfcs_errs_edger = l2_err_competitor(other_predictions["edger"]["lfc"], other=lfc_gt).assign(algorithm="EdgeR")
lfcs_errs_mast = l2_err_competitor(other_predictions["mast"]["lfc"], other=lfc_gt).assign(algorithm="MAST")

In [ ]:
def pd_l2_err(my_df):
    diff = my_df.sort_values("gene")["lfc_mean"] - lfc_gt
    error = 0.5 * (diff ** 2) ** (0.5)
    error = np.nanmean(error)
    return pd.Series(dict(error=error))

lfcs_errs_mf = (
    res_mf
    .groupby(["experiment", "sample_size", "training", "algorithm"])
    .apply(pd_l2_err)
    .reset_index()
)

lfcs_errs_iaf = (
    res_iaf
    .groupby(["experiment", "sample_size", "training", "algorithm"])
    .apply(pd_l2_err)
    .reset_index()
)

In [ ]:
all_errs = pd.concat([
    lfcs_errs_mf,
    lfcs_errs_iaf,
    lfcs_errs_deseq2,
    lfcs_errs_edger,
    lfcs_errs_mast,
], ignore_index=True)

px.box(all_errs, x="sample_size", y="error", color="algorithm")

In [ ]:
def algos_comparison(my_df, key1, other_keys):
    vals_key1 = my_df.loc[my_df["algorithm"]==key1, "error"].values
    algo1_is_better = True
    for key2 in other_keys:
        vals_other = my_df.loc[my_df["algorithm"] == key2, "error"].values
        key1_better = has_lower_mean(vals_key1, vals_other)
        if not key1_better:
            algo1_is_better = False
            break
    return key1_better

gped = all_errs.groupby("sample_size")
mf_or_iaf_better = (
    gped.apply(algos_comparison, key1="MF", other_keys=["DESeq2", "EdgeR", "MAST"]) &
    gped.apply(algos_comparison, key1="IAF", other_keys=["DESeq2", "EdgeR", "MAST"])
)
mf_better = gped.apply(algos_comparison, key1="MF", other_keys=["IAF", "DESeq2", "EdgeR", "MAST"])
iaf_better = gped.apply(algos_comparison, key1="IAF", other_keys=["MF", "DESeq2", "EdgeR", "MAST"])

In [ ]:
res_table = (
    all_errs.groupby(["sample_size", "algorithm"])
    .error.agg(dict(err_mean="mean", err_std="std"))
    .reset_index()
    .assign(
        displayed=lambda x: x.apply(
            lambda y: "{:.3f} \pm {:.3f}".format(y.err_mean, y.err_std), axis=1
        ),
        is_better=False,
        one_of_best=False,
    )
)
res_table.loc[res_table["algorithm"] == "MF", "is_better"] = mf_better.values
res_table.loc[res_table["algorithm"] == "IAF", "is_better"] = iaf_better.values
res_table.loc[res_table["algorithm"] == "IAF", "one_of_best"] = mf_or_iaf_better.values
res_table.loc[res_table["algorithm"] == "MF", "one_of_best"] = mf_or_iaf_better.values


res_table.loc[lambda x: x.one_of_best, "displayed"] = (
    res_table.loc[lambda x: x.one_of_best, "displayed"] + "^*"
)
res_table.loc[lambda x: x.is_better, "displayed"] = res_table.loc[
    lambda x: x.is_better, "displayed"
].apply(lambda x: "\mathbf{{ {} }}".format(x))

res_table.loc[:, "displayed"] = res_table.loc[:, "displayed"].apply(lambda x: "$ {} $".format(x)) 

In [ ]:
res_table.pivot(index="algorithm", columns="sample_size", values="displayed").loc[
    ["DESeq2", "EdgeR", "MAST", "MF", "IAF"]
]

## Coverage

In [ ]:
res_mf.info()

In [ ]:
print((res_mf.hdi64_high - res_mf.hdi64_low).mean())
print((res_iaf.hdi64_high - res_iaf.hdi64_low).mean())

In [ ]:
def get_coverage(my_df, low_key="hdi64_low", high_key="hdi64_high"):
    my_df = my_df.sort_values("gene")
    assert len(my_df) == n_genes
    gene_is_covered = (lfc_gt >= my_df[low_key]) & (lfc_gt <= my_df[high_key])
#     mean_cov = (gene_is_covered / (my_df[high_key] - my_df[low_key])).mean()
    mean_cov = (gene_is_covered).mean()
    return pd.Series(dict(mean_cov=mean_cov))
    

coverage_mf = (
    res_mf.groupby(["experiment", "training", "sample_size", "algorithm"])
    .apply(get_coverage, low_key="hdi64_low", high_key="hdi64_high")
    .reset_index()
#     .groupby("sample_size")
#     .agg(dict(mean_cov=["mean", "std"]))
)
coverage_iaf = (
    res_iaf.groupby(["experiment", "training", "sample_size", "algorithm"])
    .apply(get_coverage, low_key="hdi64_low", high_key="hdi64_high")
    .reset_index()
#     .groupby("sample_size")
#     .agg(dict(mean_cov=["mean", "std"]))
)

all_coverages = pd.concat([coverage_mf, coverage_iaf], ignore_index=True)

In [ ]:
px.box(all_coverages, x="sample_size", y="mean_cov", color="algorithm")

## Volcano

In [ ]:
selected_training = 0

preds_md = res_mf.loc[
    lambda x: (x.experiment == 0) & (x.training == selected_training) & (x.sample_size == 100)
].sort_values("gene")["de_proba"]

preds_iaf = res_iaf.loc[
    lambda x: (x.experiment == 0) & (x.training == selected_training) & (x.sample_size == 100)
].sort_values("gene")["de_proba"]



fig = go.Figure(
    layout=go.Layout(
        yaxis=dict(title="Estimated probabily of DE"),
        xaxis=dict(title="Ground-Truth LFC"),
    )
)
fig.add_traces(
    [
        go.Scatter(x=lfc_gt, y=np.log10(preds_md), mode="markers", name="MF"),
        go.Scatter(x=lfc_gt, y=np.log10(preds_iaf), mode="markers", name="IAF"),
#         go.Scatter(x=lfc_gt, y=np.log10(preds_mast), mode="markers"),
#         go.Scatter(x=lfc_gt, y=np.log10(preds_edger), mode="markers"),
        go.Scatter(
            x=[-0.5, -0.5], y=[-6, 0.0], mode="lines", line=dict(color="black", width=2)
        ),
        go.Scatter(
            x=[0.5, 0.5], y=[-6, 0.0], mode="lines", line=dict(color="black", width=2)
        ),
        #         go.Scatter(
        #             x=[alpha, alpha], y=[-6, 0.0], mode="lines", line=dict(color="black", width=2)
        #         ),
    ]
)

fig.show()
# iplot(fig, filename="symsim_volcano", sharing="private")

# Debug

In [ ]:
# trace1 = go.Histogram(x=dataset.X.mean(0))
# fig = go.Figure(data=[trace1])
# fig.show()

# dataset.X

# # from torch.autograd import set_detect_anomaly
# # set_detect_anomaly(True)

# iw_vae = IAVAE(
#     dataset.nb_genes, n_batch=dataset.n_batches, n_hidden=32, n_layers=1, do_h=True, n_latent=10, t=4
# )
# iw_trainer = UnsupervisedTrainer(
#     iw_vae, dataset, ratio_loss=True, k_importance_weighted=5, single_backward=False
# )
# iw_trainer.train(n_epochs=50, lr=1e-3)
# iw_trainer.train_losses

iw_vae = VAE(
    dataset.nb_genes, n_batch=dataset.n_batches, n_hidden=128, n_layers=1, n_latent=10
)
iw_trainer = UnsupervisedTrainer(
    iw_vae, dataset, ratio_loss=True, k_importance_weighted=5, single_backward=False
)
iw_trainer.train(n_epochs=50, lr=1e-3)
iw_trainer.train_losses


vae = VAE(
    dataset.nb_genes, n_batch=dataset.n_batches, n_hidden=128, n_layers=1, n_latent=10
)
trainer = UnsupervisedTrainer(
    vae, dataset, ratio_loss=True, #k_importance_weighted=5, single_backward=False
)
trainer.train(n_epochs=50, lr=1e-3)
trainer.train_losses



# iw_trainer.train_losses

# iavae = IAVAE(
#     dataset.nb_genes,
#     n_batch=dataset.n_batches,
#     n_hidden=128,
#     n_layers=1,
#     do_h=True,
#     n_latent=5,
#     t=4,
# )
# ia_trainer = UnsupervisedTrainer(iavae, dataset, ratio_loss=True)
# ia_trainer.train(n_epochs=100, lr=1e-3)

# ia_trainer.train_losses

# plt.plot(trainer.train_losses[5:], label="MF")
# plt.plot(iw_trainer.train_losses[5:], label="IW")
# plt.plot(ia_trainer.train_losses[5:], label="IAF")
# # plt.yscale("log")
# plt.legend()

# ia_trainer.test_set.marginal_ll(n_mc_samples=100, ratio_loss=True)



# evidence_mf = trainer.test_set.marginal_ll(n_mc_samples=100, ratio_loss=True)
# # evidence_iw = iw_trainer.test_set.marginal_ll(n_mc_samples=100, ratio_loss=True)
# evidence_ia = ia_trainer.test_set.marginal_ll(n_mc_samples=100, ratio_loss=True)
# print(evidence_mf, evidence_iw, evidence_ia)

In [ ]:
print(iw_trainer.test_set.marginal_ll(n_mc_samples=500, ratio_loss=True))
print(trainer.test_set.marginal_ll(n_mc_samples=500, ratio_loss=True))


In [ ]:
test_iw = iw_trainer.test_set
test_mf = trainer.test_set

In [ ]:
def get_lfc(post):
    z_iaf, labels_iaf, scales_iaf = post.get_latents(n_samples=100, other='scales', device="cpu")
    where_a = np.where(labels_iaf == 0)[0][:200]
    where_b = np.where(labels_iaf == 1)[0][:200]

    scales_a = scales_iaf[:, where_a, :]
    scales_b = scales_iaf[:, where_b, :]

    lfc = np.log2(scales_a) - np.log2(scales_b)
    lfc = lfc.reshape((-1, n_genes))
    lfc = np.array(lfc)
    return (np.abs(lfc) >= DELTA).mean(0)

de_probas_iw = get_lfc(test_iw)
de_probas_mf = get_lfc(test_mf)

In [ ]:
fig = go.Figure()
plot_pr(fig=fig, preds=de_probas_iw, y_true=is_significant_de, name='IW IAF')
plot_pr(fig=fig, preds=de_probas_mf, y_true=is_significant_de, name='MF')
fig.show()

4395.42928125 4369.418739583333 4391.585421875

The lower the better

Using very quick experiments (I am waiting for the autotune module), it looks like:

MF
- 1 layer 16 941.0715670955882
- 1 layer 64 935.0418129595588
- 1 layer 128 925.3853147977941
- 3 layers 32 972.2561259191176

IAF
- 1 layer 16 950.423221507353
- 1 layer 64 928.0418129595588
- 1 layer 128 917.7115165441177
- 3 layers 32 928.8929044117647
- t=4 1 layer 128 915.8975482536765
- t=5 1 layer 128 920.9016911764705

### Visualizing IAF posteriors

In [ ]:
# post = ia_trainer.train_set
# train_indices = post.data_loader.sampler.indices
# train_samples = np.random.permutation(train_indices)[:2000]
# post = ia_trainer.create_posterior(
#     model=iavae, gene_dataset=dataset, indices=train_samples
# )
# z_ia, labels_ia, scales_ia = post.get_latents(n_samples=500, other=True, device="cpu")

In [ ]:
# # post = trainer.train_set
# # train_indices = post.data_loader.sampler.indices
# # train_samples = np.random.permutation(train_indices)[:2000]
# post = trainer.create_posterior(model=vae, gene_dataset=dataset, indices=train_samples)
# z, labels, scales = post.get_latents(n_samples=500, other=True, device="cpu")
print((labels == labels_ia).float().mean())

In [ ]:
# for idx in [5, 10, 100, 1000, 2, 30, 542]:
#     trace1 = go.Scatter(x=z[:, idx, 0], y=z[:, idx, 1], mode="markers")
#     trace2 = go.Scatter(x=z_ia[:, idx, 0], y=z_ia[:, idx, 1], mode="markers")
#     fig = go.Figure([trace1, trace2])
#     fig.show()

In [ ]:
# from plotly.subplots import make_subplots
# from sklearn.manifold import TSNE


# fig = make_subplots(rows=1, cols=2)

# for my_z in []:
#     z_mean = my_z.mean(0)
#     z_tsne = TSNE().fit_transform(z_mean)
#     #     z_tnse = z_mean

#     fig.add_trace(
#         go.Scatter(
#             x=z_tnse[:, 0],
#             y=z_tnse[:, 1],
#             marker=dict(color=my_lbl.squeeze(), colorscale="viridis"),
#             mode="markers",
#         ),
#         row=1,
#         col=1,
#     )

# fig

In [ ]:
# labels = labels.squeeze()
# where_a = np.where(labels == 0)[0]
# where_b = np.where(labels == 1)[0]
# where_a = where_a[np.random.choice(len(where_a), size=size)]
# where_b = where_b[np.random.choice(len(where_b), size=size)]
# scales_a = scales[:, where_a, :].reshape((-1, dataset.nb_genes)).numpy()
# scales_b = scales[:, where_b, :].reshape((-1, dataset.nb_genes)).numpy()
# scales_a, scales_b = demultiply(arr1=scales_a, arr2=scales_b, factor=3)
# lfc = np.log2(scales_a) - np.log2(scales_b)

# pgs = (np.abs(lfc) >= DELTA).mean(axis=0)
# sorted_genes = np.argsort(-pgs)
# sorted_pgs = pgs[sorted_genes]
# cumulative_fdr = (1.0 - sorted_pgs).cumsum() / (1.0 + np.arange(len(sorted_pgs)))
# d = (cumulative_fdr <= Q0).sum() - 1
# pred_de_genes = sorted_genes[:d]
# is_pred_de = np.zeros_like(cumulative_fdr)
# is_pred_de[pred_de_genes] = True
# true_fdr = ((~is_significant_de) * is_pred_de).sum() / len(pred_de_genes)

# true_fdr

In [ ]:
# from sklearn.metrics import confusion_matrix

# y_preds_1d = is_pred_de.reshape((-1, dataset.nb_genes))
# n_exps = len(y_preds_1d)
# mat = confusion_matrix(is_significant_de, is_pred_de)
# mat